In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable import RunnablePassthrough 
from langchain.chains import LLMChain

In [2]:
memory = ConversationBufferMemory(return_messages=True, memory_key="history")

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

In [3]:
movie_emoji_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 영화 전문가입니다.",
        ),
        FewShotChatMessagePromptTemplate(
            example_prompt=ChatPromptTemplate.from_messages(
                [
                    ("human", "{movie}"),
                    ("ai", "{answer}"),
                ]
            ),
            examples=[
                {
                    "movie": "다크 나이트",
                    "answer": "🦇💥🤡",
                },
                {
                    "movie": "다크 나이트 라이즈",
                    "answer": "🦇💥😷",
                },
            ],
        ),
        ("human", "{movie}"),
    ]
)


def invoke_movie_chain(movie: str):
    chain = movie_emoji_prompt | chat
    result = chain.invoke({"movie": movie})
    memory.save_context({"input": movie}, {"output": result.content})
    return result

In [4]:
invoke_movie_chain("탑건")

✈️🔥🕶️

AIMessageChunk(content='✈️🔥🕶️')

In [6]:
invoke_movie_chain("대부")

🍊🔫💰

AIMessageChunk(content='🍊🔫💰')

In [7]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='탑건'),
  AIMessage(content='✈️🔥🕶️'),
  HumanMessage(content='대부'),
  AIMessage(content='🍊🔫💰')]}

In [8]:
emoji_movie_chain = RunnablePassthrough.assign(
    history=lambda _input: memory.load_memory_variables({})["history"]
) | LLMChain(
    llm=chat,
    prompt=ChatPromptTemplate.from_messages(
        [
            MessagesPlaceholder(variable_name="history"),
            ("human", "{question}"),
        ]
    ),
    memory=memory,
)

In [9]:
emoji_movie_chain.invoke({"question": "니가 ✈️🔥🕶️로 대답했던 영화가 뭐야?"})

"탑건"입니다!

{'question': '니가 ✈️🔥🕶️로 대답했던 영화가 뭐야?',
 'history': [HumanMessage(content='탑건'),
  AIMessage(content='✈️🔥🕶️'),
  HumanMessage(content='대부'),
  AIMessage(content='🍊🔫💰'),
  HumanMessage(content='니가 ✈️🔥🕶️로 대답했던 영화가 뭐야?'),
  AIMessage(content='"탑건"입니다!')],
 'text': '"탑건"입니다!'}

In [10]:
emoji_movie_chain.invoke({"question": "니가 🍊🔫💰로 대답했던 영화가 뭐야?"})

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for gpt-3.5-turbo in organization org-RmC8JUwxAKbZjSNRrAHiavnt on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing..


"대부"입니다!

{'question': '니가 🍊🔫💰로 대답했던 영화가 뭐야?',
 'history': [HumanMessage(content='탑건'),
  AIMessage(content='✈️🔥🕶️'),
  HumanMessage(content='대부'),
  AIMessage(content='🍊🔫💰'),
  HumanMessage(content='니가 ✈️🔥🕶️로 대답했던 영화가 뭐야?'),
  AIMessage(content='"탑건"입니다!'),
  HumanMessage(content='니가 🍊🔫💰로 대답했던 영화가 뭐야?'),
  AIMessage(content='"대부"입니다!')],
 'text': '"대부"입니다!'}